In [1]:
from dataset.lamah import LamaH

import numpy as np
import networkx as nx

from bokeh.io import output_file, show
from bokeh.plotting import figure, from_networkx
from bokeh.palettes import Spectral4
from bokeh.models import (BoxSelectTool, BoxZoomTool, ResetTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, TapTool, Range1d)

import pydot
from networkx.drawing.nx_pydot import graphviz_layout

/usr/local/Caskroom/miniconda/base/envs/tsl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = LamaH()
ts_qobs_df, ts_exos_dict, attribs_dict, dists_mtx, ts_qobs_mask = data.load()

In [3]:
g = nx.Graph()

nodes = list(range(len(ts_qobs_df.columns)))
attribs = [dict(ID=n) for n in ts_qobs_df.columns]
nodes_attribs = list(zip(nodes, attribs))

g.add_nodes_from(nodes_attribs)
edge_index = data.get_connectivity(method='distance',
                                   layout='edge_index')[0]
edge_index = list(zip(edge_index[0].tolist(), edge_index[1].tolist()))
g.add_edges_from(edge_index)

In [4]:
plot = figure(title='LamaH-CE Network', x_range=Range1d(-200, 12200), y_range=Range1d(-100, 1500),
              width=1900, height=400)

node_hover_tool = HoverTool(tooltips=[('ID', '@ID')])
plot.add_tools(node_hover_tool, TapTool())

graph_renderer = from_networkx(g, graphviz_layout(g, prog='dot'), scale=2, center=(0,0)) #circo

graph_renderer.node_renderer.glyph = Circle(size=10, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=10, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=10, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color='#CCCCCC', line_alpha=0.8, line_width=2)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=2)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=2)

graph_renderer.selection_policy = NodesAndLinkedEdges()
# graph_renderer.inspection_policy = EdgesAndLinkedNodes()


plot.renderers.append(graph_renderer)

output_file('./data/LamaH-CE/lamah_network.html')
show(plot)

In [24]:
node_index = dict(zip(list(ts_qobs_df.columns), list(range(len(ts_qobs_df.columns)))))
adjacencym = data.get_connectivity(method='stcn',
                                   layout='dense')

In [25]:
print(f'59 -> {node_index[59]}')
print(f'60 -> {node_index[60]}')

59 -> 55
60 -> 56


In [26]:
print(f'### DIST MTX (shape: {dists_mtx.shape}) ###')
print(f'59 -> 60: {dists_mtx[55,56]}')
print(f'59 -> 60: {dists_mtx[56,55]}')
print(f'### ADJA MTX (shape: {adjacencym.shape}) ###')
print(f'59 -> 60: {adjacencym[55,56]}')
print(f'59 -> 60: {adjacencym[56,55]}')

### DIST MTX (shape: (850, 850)) ###
59 -> 60: 144.10000610351562
59 -> 60: inf
### ADJA MTX (shape: (850, 850)) ###
59 -> 60: 0.0
59 -> 60: 0.0


In [27]:
adjacencym

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.33905265, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.9964065 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [28]:
dists_mtx

array([[ inf,  inf,  inf, ...,  inf,  inf,  inf],
       [ inf,  inf, 10.4, ...,  inf,  inf,  inf],
       [ inf,  inf,  inf, ...,  inf,  inf,  inf],
       ...,
       [ inf,  inf,  inf, ...,  inf,  0.6,  inf],
       [ inf,  inf,  inf, ...,  inf,  inf,  inf],
       [ inf,  inf,  inf, ...,  inf,  inf,  inf]], dtype=float32)